In [1]:
using VariableHeightMomentumControl
using RigidBodyDynamics
using RigidBodyDynamics.OdeIntegrators
using RigidBodyTreeInspector
using StaticArrays
using DrakeVisualizer
using ValkyrieRobot
using GeometryTypes
using ColorTypes

In [2]:
# Create mechanism
val = Valkyrie()
mechanism = val.mechanism;

In [3]:
# Add sole frame
ankleRoll = first(filter(j -> j.name == "leftAnkleRoll", joints(mechanism)))
soleFrame = CartesianFrame3D("l_foot_sole")
soleToFoot = Transform3D(soleFrame, ankleRoll.frameAfter, SVector(0.067, 0., -0.09))
add_body_fixed_frame!(mechanism, soleToFoot);

In [4]:
# Reroot at the left foot and remove fixed joints
pelvis = first(filter(b -> name(b) == "pelvis", bodies(mechanism)))
pelvisToWorld = first(filter(j -> j.name == "pelvis_to_world", joints(mechanism)))
leftFoot = first(filter(b -> name(b) == "leftFoot", bodies(mechanism)))
leftFootToWorld = Joint("leftFootToWorld", Fixed{Float64}())
world = root_body(mechanism)
attach!(mechanism, world, leftFootToWorld, Transform3D(leftFootToWorld.frameBefore, default_frame(world), -translation(soleToFoot)), leftFoot)
remove_joint!(mechanism, pelvisToWorld)

In [5]:
# remove_fixed_tree_joints!(mechanism); # currently causes vis issues

In [6]:
# Create visualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window(); sleep(1)
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]))

In [7]:
# Add box to stand on
delete!(vis[:box])
box = GeometryData(HyperRectangle(Vec(-0.4, -0.08, -0.1), Vec(0.8, 0.6, 0.1)), RGBA(0.4, 0.4, 0.4, 0.8))
addgeometry!(vis[:box], box)

Visualizer with path prefix Symbol[:valkyrie, :box] using LCM LCMCore.LCM(Ptr{Void} @0x0000000003f9a020, "", RawFD(61), LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#9{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#9{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT, DrakeVisualizer.handle_msg), Ptr{Void} @0x0000000004d2ee00)])

In [17]:
# Create controller
centroidalFrame = CartesianFrame3D("centroidal")
# pelvisVertex = findfirst(v -> name(vertex_data(v)) == "pelvis", tree(mechanism))
# stanceFootVertex = findfirst(v -> name(vertex_data(v)) == "world", tree(mechanism))
# stanceLegPath = path(pelvisVertex, stanceFootVertex)
stanceLegPath = path(mechanism, pelvis, root_body(mechanism))
stanceLegJoints = collect(stanceLegPath)
positionControlledJoints = collect(filter(j -> j ∉ stanceLegJoints, joints(mechanism)))
controller = VariableHeightMomentumController(mechanism, positionControlledJoints, centroidalFrame, soleFrame);

In [18]:
function passive_dynamics!(vd::AbstractArray, t, state)
    dynamics!(result, state)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

passive_dynamics! (generic function with 1 method)

In [19]:
function controlled_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    torques = control(controller, t, state)
    dynamics!(result, state, torques)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

controlled_dynamics! (generic function with 1 method)

In [20]:
const τ = Vector{Float64}(num_velocities(mechanism))
function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    τ[:] = velocity(state)
    scale!(τ, -1.0)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

damped_dynamics! (generic function with 1 method)

In [21]:
# Simulation
state = MechanismState{Float64}(mechanism)
result = DynamicsResult{Float64}(mechanism)
# dyn! = damped_dynamics!
dyn! = controlled_dynamics!
integrator = MuntheKaasIntegrator(dyn!, runge_kutta_4(Float64), DrakeVisualizerSink(vis));

In [45]:
zero!(state)
θroll = 0.16
θstanceKnee = 1.
configuration(state, findjoint(mechanism, "leftAnkleRoll"))[:] = θroll
configuration(state, findjoint(mechanism, "leftHipRoll"))[:] = -θroll
configuration(state, findjoint(mechanism, "leftKneePitch"))[:] = θstanceKnee
configuration(state, findjoint(mechanism, "leftHipPitch"))[:] = -θstanceKnee
configuration(state, findjoint(mechanism, "rightHipRoll"))[:] = -0.1
configuration(state, findjoint(mechanism, "leftShoulderRoll"))[:] = -1.
configuration(state, findjoint(mechanism, "rightShoulderRoll"))[:] = 1.
configuration(state, findjoint(mechanism, "leftShoulderYaw"))[:] = 1.2
configuration(state, findjoint(mechanism, "rightShoulderYaw"))[:] = 1.2

1.2

In [46]:
xd = 1.0 + (1.4 - 1.0) * rand()
initialize_state!(state, xd, positionControlledJoints)
settransform!(vis, state)
for joint in positionControlledJoints
    set_desired_configuration!(controller, joint, configuration(state, joint))
end

In [47]:
@time integrate(integrator, state, 3., 1e-3, maxRealtimeRate = Inf)

 19.850667 seconds (51.25 M allocations: 3.659 GiB, 4.79% gc time)
